In [ ]:
"""
Файл достает из базы данных данные из таблиц user, post и feed
Данные в таблице feed сильно смещены: соотношение рекций ползователя лайк/не_лайк ~ 1/10.
Поэтому искуственно сбалансируем данные вытащив по 1млн каждой реакции.

"""

In [1]:
import pandas as pd
from tqdm import tqdm
import uri
conn_uri = uri.conn_uri


In [2]:
def get_user(conn_uri_):
    return pd.read_sql("""SELECT * FROM public.user_data """ , conn_uri_ )

def get_post(conn_uri_):
    return pd.read_sql("""SELECT * FROM public.post_text_df""", conn_uri_ )
     
def get_feed(conn_uri_, n_feed): 
    return pd.read_sql(f"""SELECT * FROM public.feed_data limit {n_feed}""", conn_uri_ )   

def get_feed_2(conn_uri_, n_feed): 
    return pd.read_sql(f"""SELECT * FROM public.feed_data  where  action = 'view' limit {n_feed}""", conn_uri_ )   

def get_feed_like(conn_uri_, n_feed): 
    return pd.read_sql(f"""SELECT * FROM public.feed_data  where  (action = 'view' and target= 1) limit {n_feed}""", conn_uri_ )   

def get_feed_not_like(conn_uri_, n_feed): 
    return pd.read_sql(f"""SELECT * FROM public.feed_data  where  (action = 'view'and target= 0) limit {n_feed}""", conn_uri_ )   


In [ ]:
# 1 млн лайкоы
feed_like = get_feed_like(conn_uri, 1000000)
feed_like.to_csv('feed_like1000k.csv', index=False) 

In [ ]:
#Если проблемы со скачиванием можно качать чанками
#feed_like = pd.DataFrame()
#for chunk in tqdm(feed_like_chank):
#    feed_like = pd.concat([feed_like, chunk])

In [ ]:
# 1 млн пустой реакции
feed_not_like = get_feed_not_like(conn_uri, 1000000)
feed_not_like.to_csv('feed_not_like1000k.csv', index=False) 

In [ ]:
#Если проблемы со скачиванием можно качать чанками
#feed_not_like = pd.DataFrame()
#for chunk in tqdm(feed_not_like_chank):
#    feed_not_like = pd.concat([feed_not_like, chunk])

In [ ]:
#датасет 50/50 лайки/просмотры
feed50_50 = pd.concat((feed_like, feed_not_like), axis =0).reset_index() 
feed50_50.to_csv('feed50_50_2mil',index=False)

In [ ]:
post_save = get_post(conn_uri) #чистый пост
user_save = get_user(conn_uri) #чистый юзер

In [ ]:
post_save.to_csv('post.csv', index=False)   
user_save.to_csv('user.csv',index=False)